In [1]:
import spatioloji_s as sj
import numpy as pd
import pandas as pd
import psutil
import os
import matplotlib.pyplot as plt
import cv2

In [2]:
mem = psutil.virtual_memory()
print(f"Total memory available: {mem.total / 1024**3:.2f} GB")
cpus = os.cpu_count()
print(f"Total cpu cores available: {cpus}")
print(f"current work directory is {os.getcwd()}")

Total memory available: 251.40 GB
Total cpu cores available: 64
current work directory is /carc/scratch/projects/amitra2016502/spatioloji_s/example


In [3]:
# Load your spatioloji object
sp = sj.data.spatioloji.from_pickle('my_qc_output/data/filtered_spatioloji.pkl')



Loading spatioloji from: my_qc_output/data/filtered_spatioloji.pkl
✓ Loaded: 25,368 cells × 960 genes


In [4]:
# Normalize data
sj.processing.normalize_total(sp, target_sum=1e4, inplace=True)
sj.processing.log_transform(sp, layer='normalized_counts', inplace=True)

# Select highly variable genes
# # of HVG may be less than n_top_genes because we have other cut-off values to filter
sj.processing.highly_variable_genes(sp, n_top_genes=2000, inplace=True)

# Dimensionality reduction
sj.processing.pca(sp, n_comps=50, inplace=True)
sj.processing.umap(sp, use_pca=True, inplace=True)

# Clustering
sj.processing.leiden_clustering(sp, resolution=0.5, inplace=True, layer='normalized_counts')


Normalizing expression to target sum: 10,000
  Counts per cell - mean: 179, median: 140
  ✓ Normalized 25,368 cells × 960 genes
✓ Added layer 'normalized_counts' (dense, 185.8 MB)

Applying log transformation (base=e)
  ✓ Transformed 25,368 cells × 960 genes
✓ Added layer 'log_normalized' (dense, 185.8 MB)

Identifying highly variable genes (method=seurat)
  Using Seurat method (n_top_genes=2000)
  ✓ Selected 380 highly variable genes
    Mean expression range: 0.02 - 2.63

Computing PCA (n_comps=50)
  Using 380 highly variable genes
  Running PCA...
  ✓ PCA complete
    Variance explained by first 10 PCs: 21.9%
    Variance explained by all 50 PCs: 39.6%


/users/jiwang1/.conda/envs/spatioloji/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2025-12-27 18:47:29.394733: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.
2025-12-27 18:47:29.395065: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-12-27 18:47:29.444067: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI AVX512_BF16 AVX512_FP16 AVX_VNNI AMX


UMAP (n_components=2, n_neighbors=15, min_dist=0.1)
  Using first 50 PCs as input
  Running UMAP...


/users/jiwang1/.conda/envs/spatioloji/lib/python3.12/site-packages/umap/umap_.py:1952: UserWarning: n_jobs value 1 overridden to 1 by setting random_state. Use no seed for parallelism.
  warn(


  ✓ UMAP complete

Leiden clustering (resolution=0.5, n_neighbors=15)
  Using 380 highly variable genes (out of 960 total)

Leiden clustering (resolution=0.5, n_neighbors=15)
  Computing PCA (n_pcs=50)...
    Variance explained: 65.9%
  Building k-NN graph...
  Running Leiden algorithm...
  ✓ Found 7 clusters
    Cluster sizes - min: 1326, max: 7835, mean: 3624.0


In [5]:
sp._gene_meta

,mt,ribo,NegProbe,QC_gene_pass,qc_filtered,means,variances,dispersions,mean_bin,dispersions_norm,highly_variable
gene,,,,,,,,,,,
AATK,False,False,False,True,True,0.018409,0.021460,1.165745,0,-1.059713,False
ABL1,False,False,False,True,True,0.058578,0.075172,1.283280,0,-0.283110,False
ABL2,False,False,False,True,True,0.094371,0.126462,1.340049,0,0.091988,False
ACE,False,False,False,True,True,0.033152,0.042539,1.283138,0,-0.284044,False
ACE2,False,False,False,True,True,0.032088,0.039336,1.225898,0,-0.662259,False
...,...,...,...,...,...,...,...,...,...,...,...
XCL1,False,False,False,True,True,0.045963,0.052208,1.135855,0,-1.257212,False
XCL2,False,False,False,True,True,0.018448,0.021341,1.156765,0,-1.119047,False
YBX3,False,False,False,True,True,0.206402,0.327786,1.588098,0,1.730963,True
